In [2]:
import pandas as pd
import numpy as np
import requests
from binance.client import Client
from secrets import api_key, api_secret
client = Client(api_key, api_secret)

"""
OHLCV:
    Timestamp	Epoch timestamp in milliseconds. You can learn more about timestamps, including how to convert them to human readable form, here. 
    Open	Opening price of the time interval in quote currency (For BTC/USD, the price would be USD).
    High	Highest price reached during time interval, in quote currency.
    Low	Lowest price reached during time interval, in quote currency.
    Close	Closing price of the time interval, in the quote currency. 
    Volume	Quantity of asset bought or sold, displayed in base currency.
"""
'''
#### If want stream data #####
# start aggregated trade websocket for BNBBTC
def process_message(msg):
    print("message type: {}".format(msg['e']))
    print(msg)
    # do something
from binance.websockets import BinanceSocketManager
bm = BinanceSocketManager(client)
bm.start_aggtrade_socket('BNBBTC', process_message)
bm.start()
'''
print('ok')

ok


In [96]:
import numpy as np #pip install numpy
from tqdm import tqdm #pip install tqdm
from binance.client import Client #pip install python-binance
import pandas as pd #pip install pandas
from datetime import datetime
import random
import time

START_TIME = '28 Mar, 2019'
END_TIME = '1 Jun, 2020'

ratios = ['BTC']

for ratio in ratios:
    print(f'Gathering {ratio} data...')
    data = client.get_historical_klines(symbol=f'{ratio}USDT',interval=Client.KLINE_INTERVAL_1DAY,start_str=START_TIME,end_str=END_TIME)
    cols = ['time','Open','High','Low','Close','Volume','CloseTime','QuoteAssetVolume','NumberOfTrades','TBBAV','TBQAV','null']
    df = pd.DataFrame(data,columns=cols)
    
for col in df.columns:
    if col != 'time':
        df[col] = df[col].astype(np.float64)
  
df = df.reset_index(drop=True)

#convert binance timestamp to datetime
for i in tqdm(range(len(df))):
    df['time'].iloc[i] = datetime.fromtimestamp(int(df['time'].iloc[i]/1000))
    df['CloseTime'].iloc[i] = datetime.fromtimestamp(int(df['CloseTime'].iloc[i]/1000))

Gathering BTC data...
  0%|          | 0/432 [00:00<?, ?it/s]c:\Users\ennio.junior\Documents\trading_algorithm\venv\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 432/432 [00:00<00:00, 1167.75it/s]


In [98]:
class Trading():
    def reset(self, currency):
        self.balance = {
            'BUSD': 1000000,
            currency: 0
        }
        self.currency = currency
        self.calls = []
        self.step = 1
        self.iloc_time = 0 #initial analysis point
        self.start_time = self.k_historical['time'].iloc[0]
        self.end_time = self.k_historical['time'].iloc[-1]

    def next_step(self):
        actual_df = self.k_historical.iloc[:self.step,:]
        self.strategy(actual_df)
        

    def call_buy(self, timestamp, price, amount, quote, base):
        total_transaction = price * amount
        fee = self.trading_fee_multiplier * total_transaction

        self.balance[base] = self.balance[base] - total_transaction
        self.balance[quote] += amount #TO DO: add amount_bought considering fee in balance

        self.calls.append({
            'timestamp':timestamp,
            'call':'BUY',
            'price':price,
            'amount':amount, #TO DO: add amount_bought considering fee
            'quote':quote,
            'base':base
        })
    def call_sell(self, timestamp, price, amount, quote, base):
        total_transaction = price * amount
        fee = self.trading_fee_multiplier * total_transaction
        
        self.balance[base] = self.balance[base] + total_transaction #TO DO: add amount_bought considering fee in balance
        self.balance[quote] -= amount
        
        self.calls.append({
            'timestamp':timestamp,
            'call':'SELL',
            'price':price,
            'amount':amount, #TO DO: add amount_bought considering fee
            'quote':quote,
            'base':base
        })
        

    def check_fund_enough(self, amount):
        if self.balance[self.currency] > amount:
            return True
        return False

    
    def fit(self):
        while self.step <= self.k_historical.shape[0]:
            self.iloc_time = self.step - 1
            self.next_step()
            self.step +=1
            

class StrategySMA(Trading):
    def __init__(self,df,currency):
        self.trading_fee_multiplier = .99925
        self.k_historical = df
        self.reset(currency)

    def strategy(self, actual_df):
        balance = self.balance
        close_price = actual_df['Close']
        timeline = actual_df['time']
        low_period = 9
        high_period = 30
        self.SMA_LOW = self.compute_sma(close_price,low_period).fillna(0)
        self.SMA_HIGH = self.compute_sma(close_price,high_period).fillna(0)

        if not self.check_fund_enough(0) and self.step > high_period:
            if self.SMA_LOW[self.iloc_time] > self.SMA_HIGH[self.iloc_time] and self.SMA_LOW[self.iloc_time-1] > self.SMA_HIGH[self.iloc_time-1]:
                self.call_buy(
                    timestamp=timeline[self.iloc_time],
                    price=close_price[self.iloc_time],
                    amount=1,
                    quote='BTC',
                    base='BUSD')
        
        if self.check_fund_enough(0) and self.step > high_period:
            if self.SMA_LOW[self.iloc_time] < self.SMA_HIGH[self.iloc_time]:
                self.call_sell(
                    timestamp=timeline[self.iloc_time],
                    price=close_price[self.iloc_time],
                    amount=1,
                    quote='BTC',
                    base='BUSD'
                )


    @staticmethod
    def compute_sma(data, window):
        sma = data.rolling(window=window).mean()
        return sma


In [99]:
run = StrategySMA(df,'BTC')
run.fit()

In [100]:
prices = df['Close']
timeline = df['time']
buys = np.array([(buy['timestamp'],buy['price']) for buy in run.calls if buy['call'] == 'BUY'])
sells = np.array([(sell['timestamp'],sell['price']) for sell in run.calls if sell['call'] == 'SELL'])
low_period = 9
high_period = 30
run.balance

{'BUSD': 1004979.8, 'BTC': 0}

In [111]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Closed price",
    mode="lines", x=timeline, y=prices,
))
fig.add_trace(go.Scatter(
    name="MA_high",
    mode="lines", x=timeline[high_period:], y=run.SMA_HIGH[high_period:],
))
fig.add_trace(go.Scatter(
    name="MA_low",
    mode="lines", x=timeline[low_period:], y=run.SMA_LOW[low_period:],
))
fig.add_trace(go.Scatter(
    name="Buy",
    mode="markers", x=buys[:,0], y=buys[:,1],
    marker_symbol="circle",
    marker=dict(
            color='green',
            size=8)
))
fig.add_trace(go.Scatter(
    name="Sell",
    mode="markers", x=sells[:,0], y=sells[:,1],
    marker_symbol="circle",
    marker=dict(
            color='red',
            size=8)
))
fig.show()